# Exploring the housing market of Saint Pietersburg

The data set for this research is the Yandex.Realty archive of ads for the apartments sal in St. Petersburg and neighboring settlements. 

The main goal of this research is to understand what affects the realty prices. The outcome of the research will be used to build an automated system that will track anomalies and fraudulent activity.

There are two types of data available for each apartment for sale. The first type is the data that was entered by the user, the second type is the data that obtained automatically based on cartography. For example, the distance to the center, airport, nearest park and so on.

The table contains the following data:

* 'airports_nearest` — distance to the nearest airport in meters (m)
* 'balcony` — number of balconies
* 'ceiling_height` — ceiling height (m)
* 'cityCenters_nearest` - distance to the city center (m)
* 'days_exposition` — how many days the ad was placed (from publication to removal)
* 'first_day_exposition` — date of publication
* 'floor` - floor
* 'floors_total` — total floors in the house
* 'is_apartment` - apartments (boolean type)
* 'kitchen_area` — kitchen area in square meters (m2)
* 'last_price` - price at the time of withdrawal from publication
* 'living_area` — living area in square meters(m2)
* 'locality_name` — name of the locality
* 'open_plan` — free layout (boolean type)
* 'parks_around3000` — the number of parks within a 3 km radius
* 'parks_nearest` — distance to the nearest park (m)
* 'ponds_around3000` — number of reservoirs within a radius of 3 km
* 'ponds_nearest` — distance to the nearest reservoir (m)
* 'rooms` — number of rooms
* 'studio` — studio apartment (boolean type)
* 'total_area` — the area of the apartment in square meters (m2)
* 'total_images` — the number of photos of the apartment in the ad

The research includes the following stages:

1. Data investigation.
2. Data cleanup.
3. Data analysis.
4. Follow up.

# Data investigation

First thing to do is to load tha data and check it's shape.

In [89]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [90]:
# libraries import
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
!pip install pymystem3==0.1.10
from pymystem3 import Mystem
m = Mystem()
from matplotlib import pyplot as plt


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [91]:
# data import
df = pd.read_csv("/content/drive/MyDrive/da_portfolio/real_estate_data.csv", sep="\t") 
display(df.head(10))
display(df.info())

,total_images,last_price,total_area,first_day_exposition,rooms,ceiling_height,floors_total,living_area,floor,is_apartment,...,kitchen_area,balcony,locality_name,airports_nearest,cityCenters_nearest,parks_around3000,parks_nearest,ponds_around3000,ponds_nearest,days_exposition
0,20,13000000.0,108.00,2019-03-07T00:00:00,3,2.70,16.0,51.00,8,NaN,...,25.00,NaN,Санкт-Петербург,18863.0,16028.0,1.0,482.0,2.0,755.0,NaN
1,7,3350000.0,40.40,2018-12-04T00:00:00,1,NaN,11.0,18.60,1,NaN,...,11.00,2.0,посёлок Шушары,12817.0,18603.0,0.0,NaN,0.0,NaN,81.0
2,10,5196000.0,56.00,2015-08-20T00:00:00,2,NaN,5.0,34.30,4,NaN,...,8.30,0.0,Санкт-Петербург,21741.0,13933.0,1.0,90.0,2.0,574.0,558.0
3,0,64900000.0,159.00,2015-07-24T00:00:00,3,NaN,14.0,NaN,9,NaN,...,NaN,0.0,Санкт-Петербург,28098.0,6800.0,2.0,84.0,3.0,234.0,424.0
4,2,10000000.0,100.00,2018-06-19T00:00:00,2,3.03,14.0,32.00,13,NaN,...,41.00,NaN,Санкт-Петербург,31856.0,8098.0,2.0,112.0,1.0,48.0,121.0
5,10,2890000.0,30.40,2018-09-10T00:00:00,1,NaN,12.0,14.40,5,NaN,...,9.10,NaN,городской посёлок Янино-1,NaN,NaN,NaN,NaN,NaN,NaN,55.0
6,6,3700000.0,37.30,2017-11-02T00:00:00,1,NaN,26.0,10.60,6,NaN,...,14.40,1.0,посёлок Парголово,52996.0,19143.0,0.0,NaN,0.0,NaN,155.0
7,5,7915000.0,71.60,2019-04-18T00:00:00,2,NaN,24.0,NaN,22,NaN,...,18.90,2.0,Санкт-Петербург,23982.0,11634.0,0.0,NaN,0.0,NaN,NaN
8,20,2900000.0,33.16,2018-05-23T00:00:00,1,NaN,27.0,15.43,26,NaN,...,8.81,NaN,посёлок Мурино,NaN,NaN,NaN,NaN,NaN,NaN,189.0
9,18,5400000.0,61.00,2017-02-26T00:00:00,3,2.50,9.0,43.60,7,NaN,...,6.50,2.0,Санкт-Петербург,50898.0,15008.0,0.0,NaN,0.0,NaN,289.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23699 entries, 0 to 23698
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   total_images          23699 non-null  int64  
 1   last_price            23699 non-null  float64
 2   total_area            23699 non-null  float64
 3   first_day_exposition  23699 non-null  object 
 4   rooms                 23699 non-null  int64  
 5   ceiling_height        14504 non-null  float64
 6   floors_total          23613 non-null  float64
 7   living_area           21796 non-null  float64
 8   floor                 23699 non-null  int64  
 9   is_apartment          2775 non-null   object 
 10  studio                23699 non-null  bool   
 11  open_plan             23699 non-null  bool   
 12  kitchen_area          21421 non-null  float64
 13  balcony               12180 non-null  float64
 14  locality_name         23650 non-null  object 
 15  airports_nearest   

None

Now the data is loaded and I can see that not all data can be used for analysis the way it is now: the number of rows in columns is not equal, some cells have `NaN` as a value, also there is a need to check the data for duplicates. 





# Data cleanup

I will start with checking for duplicates.

In [92]:
# converting ;ocality to the lowercase
df["locality_name"] = df["locality_name"].str.lower()

# checking for duplicates in the wholde dataframe
print("The number of duplicates is:", df.duplicated().sum())

The number of duplicates is: 0


Now that I've confirmed the absence of duplicates, I can proceed with the data cleanup process. My next task is to address any discrepancies in the number of rows and columns.

Usually, an unequal number of rows indicates that some cells lack the appropriate data. To resolve this, I will assess the number of such cells per row and examine the column names to ensure that everything is in order.

In [93]:
# checking the columns' names
df.columns

Index(['total_images', 'last_price', 'total_area', 'first_day_exposition',
       'rooms', 'ceiling_height', 'floors_total', 'living_area', 'floor',
       'is_apartment', 'studio', 'open_plan', 'kitchen_area', 'balcony',
       'locality_name', 'airports_nearest', 'cityCenters_nearest',
       'parks_around3000', 'parks_nearest', 'ponds_around3000',
       'ponds_nearest', 'days_exposition'],
      dtype='object')

No spaces, no camelcase. All fine.

In [94]:
# checking the amount of  null in the column
df.isnull().sum()

total_images                0
last_price                  0
total_area                  0
first_day_exposition        0
rooms                       0
ceiling_height           9195
floors_total               86
living_area              1903
floor                       0
is_apartment            20924
studio                      0
open_plan                   0
kitchen_area             2278
balcony                 11519
locality_name              49
airports_nearest         5542
cityCenters_nearest      5519
parks_around3000         5518
parks_nearest           15620
ponds_around3000         5518
ponds_nearest           14589
days_exposition          3181
dtype: int64

Upon closer inspection, it appears that the dataset contains a significant number of null values - almost every column has some. At this point, I must determine how to handle these values, beginning with identifying which columns will be necessary for further analysis.

Based on my initial assessment, it seems that the following columns will be required:

* `ceiling_height`
* `floors_total`
* `locality_name`
* `cityCenters_nearest`
* `days_exposition`
* `living_area`
* `kitchen_area`

So I will run the data cleanup only on them, avoiding doing anything to the other columns.


**Ceiling height** (`ceiling_height`): it should be noted that the current standard for new and old buildings alike is approximately 2.5 to 2.8 meters. However, there is an exception for "stalinka" buildings, where the ceiling height can exceed 3 meters. Unfortunately, since the table does not provide any information on the type or year of construction, there is no way to determine which category the gaps belong to. As a result, the median value of the `ceiling_height` category will be utilized to fill in these gaps.


In [95]:
# checking the median value
print("The median is:", df["ceiling_height"].median(), "meters")

The median is: 2.65 meters


In [96]:
# filling the gaps with median value
df["ceiling_height"].fillna((df["ceiling_height"].median()), inplace=True)

**Total floors in the house** (`floors_total`): it should be noted that filling in the gaps for this column requires information on the year or type of construction. Additionally, the presence of the `floor` parameter further complicates the task: while the `floors_total` gap can be filled with a median value of 9, the `floor` column may indicate that the client lives on the 13th floor, for example.

To simplify this process, I've calculated the percentage of omissions in the `floors_total` column, which amounts to only 0.36% of the total number of rows. Therefore, it should be safe to delete the rows with missing values.

In order to facilitate the use of certain functions on the column values, I will need to change the data type to numeric.


In [97]:
# deleting NaN
df = df.dropna(subset=["floors_total"])

# changing the cityCenters_nearest datatype to 'int'
df["floors_total"] = df["floors_total"].astype("int")


**The name of the locality** (`locality_name`): the number of rows with gaps is relatively small. Unfortunately, there are no parameters available to use for filling in these gaps. As a result, it will be necessary to delete the rows with missing values.


In [98]:
# deleting NaN
df = df.dropna(subset=["locality_name"])

**Proximity to the center** (`citycenter_nearest`): measures proximity to the center, contains gaps in the data due to its inapplicability to certain settlements. For instance, some small villages and towns do not have a defined "city center", as there is little distinction between the outskirts and the center, and distance from the center does not affect property prices in these areas.

However, for towns and villages that are satellite settlements of a larger city, it may be possible to indicate the distance from the main city center instead of the settlement center itself.


In [99]:
# I start with the list of the settlments
df["locality_name"].unique()

array(['санкт-петербург', 'посёлок шушары', 'городской посёлок янино-1',
       'посёлок парголово', 'посёлок мурино', 'ломоносов', 'сертолово',
       'петергоф', 'пушкин', 'деревня кудрово', 'коммунар', 'колпино',
       'поселок городского типа красный бор', 'гатчина', 'поселок мурино',
       'деревня фёдоровское', 'выборг', 'кронштадт', 'кировск',
       'деревня новое девяткино', 'посёлок металлострой',
       'посёлок городского типа лебяжье',
       'посёлок городского типа сиверский', 'поселок молодцово',
       'поселок городского типа кузьмоловский',
       'садовое товарищество новая ропша', 'павловск',
       'деревня пикколово', 'всеволожск', 'волхов', 'кингисепп',
       'приозерск', 'сестрорецк', 'деревня куттузи', 'посёлок аннино',
       'поселок городского типа ефимовский', 'посёлок плодовое',
       'деревня заклинье', 'поселок торковичи', 'поселок первомайское',
       'красное село', 'посёлок понтонный', 'сясьстрой', 'деревня старая',
       'деревня лесколово', '

In [100]:
# checking the amount of items in each category
df["locality_name"].value_counts()

санкт-петербург                    15651
посёлок мурино                       520
посёлок шушары                       439
всеволожск                           398
пушкин                               369
                                   ...  
поселок каложицы                       1
посёлок платформа 69-й километр        1
поселок почап                          1
поселок гончарово                      1
посёлок дзержинского                   1
Name: locality_name, Length: 364, dtype: int64

Although the dataset contains many settlements, some of them only have one value associated with them. Additionally, from the settlement names, it is clear that the majority of them are located in the vicinity of Saint Petersburg. The size of each settlement can be inferred from its name, with categories ranging from "village" to "settlement" to "urban-type settlement". For cities, no such distinctions are necessary, and they are simply identified by their names.

To facilitate analysis of the data, I plan to extract only the locality names and apply lemmatization to categorize them appropriately. 


In [101]:
# lemmatization with categorizing function
def lemm(liveplace):
    lemma = ' '.join(m.lemmatize(liveplace))
    if "поселок город тип" in lemma:
        return lemma.replace("посёлок город тип", '')
    elif "деревня" in lemma:
        return lemma.replace("деревня", '')
    elif "поселок" in lemma:
        return lemma.replace("поселок", '')
    elif "село" in lemma:
        return lemma.replace("село", '')
    elif "товарищество" in lemma:
         return lemma.replace("товарищество", '')
    else:
        pass
    return lemma

# applying to the df and add the result to locality_name_lem
df["locality_name_lem"] = df["locality_name"].apply(lemm)


In [102]:
# checking the results
df["locality_name_lem"].value_counts()

санкт-петербург \n                     15651
   муриный \n                            552
   шушары \n                             439
всеволожск \n                            398
пушкин \n                                369
                                       ...  
   пельгора \n                             1
   каложица \n                             1
   платформа   69 - й   километр \n        1
   почап \n                                1
   дзержинский \n                          1
Name: locality_name_lem, Length: 325, dtype: int64

Data check by category.

In [103]:
# data check by category
df.groupby("locality_name_lem")["cityCenters_nearest"].agg([min, max, sum])

,min,max,sum
locality_name_lem,,,
агалатово \n,NaN,NaN,0.0
александровский \n,27352.0,27584.0,54936.0
алексеевка \n,NaN,NaN,0.0
аннино \n,NaN,NaN,0.0
аро \n,NaN,NaN,0.0
...,...,...,...
сосновый бор \n,NaN,NaN,0.0
сясьстрой \n,NaN,NaN,0.0
тихвин \n,NaN,NaN,0.0


Upon examining the table, it becomes evident that gaps in the data are primarily related to smaller settlements that do not have a defined `distance` parameter from the center. For larger cities, such as St. Petersburg and Pavlovsk, the `distance` parameter is available.

The `sum` column indicates that for certain categories, there is no data available for the entire category, making it impossible to calculate a median value.

Filling in the median value for the entire column is not a viable solution, as in some cases the median value may exceed the settlement diameter.

Fortunately, the percentage of rows with missing values in this column amounts to only 1.3% of the total number of rows. As such, it is more convenient to delete the rows with missing values for the sake of future analysis.


In [104]:
# deleting the null values
df.dropna(subset=["cityCenters_nearest"], inplace=True)

#checking the duplicates
print("The number of duplicates is:", df["cityCenters_nearest"].isnull().sum())

The number of duplicates is: 0


**Ad placement period** (`days_exposition`) is affected by still active ads or ads that have been published for less than a day. Therefore, I am unable to fill these gaps with zeros or any other value. While the presence of these gaps may affect the analysis, I choose to delete them from the table to avoid any misinterpretation. 



In [105]:
# deleting the null values
df['days_exposition'] = df['days_exposition'].dropna()

**Living area** (`living_area`) column indicates the total area of rooms that are intended for living. To fill the gaps in this column, I will investigate the correlation between the living area of other apartments (with no gaps) and the number of rooms they have. After analyzing the data, I will fill in the gaps with the median value to ensure consistency in the data.


In [106]:
# checking the number of rooms
df['rooms'].value_counts().to_frame()

,rooms
2,5888
1,5852
3,4667
4,1051
5,300
0,117
6,101
7,57
8,12
9,8


The data shows that 0.2% of rooms have a count of 0, while 2% of the total number of rooms are apartments with 5 or more rooms. It appears that some mistakes may have occurred during data collection. For example, 0 may indicate studio apartments where the living room is combined with the kitchen, while someone may have mistakenly counted the toilet, kitchen, and other areas as "rooms," resulting in many 5+ room apartments. Alternatively, someone may be selling an entire multi-story house. 

Although the amount of such data is not statistically significant, it can be considered outliers. Therefore, I have chosen to remove this data to avoid difficulties during further analysis. 



In [107]:
# removing all lines with rooms more than 5
df = df.drop(df[df.rooms >= 5].index)

# removing all lines with rooms  0
df = df.drop(df[df.rooms == 0].index)

# replacing with median by 'rooms' category
df['living_area']=df.groupby('rooms')['living_area'].transform(lambda x: x.fillna(x.median()))

# updating data type to ('int')
df['living_area'] = df['living_area'].astype('int')

**Kitchen area** (`kitchen_area`) column represents the area of the kitchen and is not part of the living area. The non-living area includes the hallway, bathroom, toilet, etc. As we do not have data for all these areas, we cannot calculate the kitchen area by subtracting the living area from the total area and then subtracting the areas of the hallway, balcony, etc. Therefore, it is more convenient to fill in the missing values with the median value for the respective category.

Additionally, there are studios in the table where there may not be a kitchen at all. Therefore, I will first investigate the impact of studios on the entire table and then fill in the missing values with the median value.

In [108]:
# checking if there are any studios in the table with a specified kitchen area
df.query('studio == True and kitchen_area>=1')

,total_images,last_price,total_area,first_day_exposition,rooms,ceiling_height,floors_total,living_area,floor,is_apartment,...,balcony,locality_name,airports_nearest,cityCenters_nearest,parks_around3000,parks_nearest,ponds_around3000,ponds_nearest,days_exposition,locality_name_lem


There are no such studios. Let's find out how many studios are there in the table.


In [109]:
# general amount of studio apt
df["studio"].value_counts()

False    17453
True         5
Name: studio, dtype: int64

Since the number of studios is not statistically significant, I will remove them from the analysis and will not fill in missing values for this category.

In [110]:
# deleting studios
df = df.drop(df[df.studio == True].index)

# checking median
df['kitchen_area'].median()

9.5

In [111]:
# filling the gaps with median
df['kitchen_area'].fillna(df['kitchen_area'].median(), inplace=True)

# changing data type ('int')
df['kitchen_area'] = df['kitchen_area'].astype('int')

Now the data cleanup is done and I am cheking reults.

In [112]:
# are there any gaps lefrt in ceiling_height, floors_total, locality_name, cityCenters_nearest, days_expositio
df.isnull().sum()

total_images                0
last_price                  0
total_area                  0
first_day_exposition        0
rooms                       0
ceiling_height              0
floors_total                0
living_area                 0
floor                       0
is_apartment            15376
studio                      0
open_plan                   0
kitchen_area                0
balcony                  8442
locality_name               0
airports_nearest           22
cityCenters_nearest         0
parks_around3000            0
parks_nearest            9779
ponds_around3000            0
ponds_nearest            8808
days_exposition          2179
locality_name_lem           0
dtype: int64

In [113]:
# if table is visually ok
df.head(5)

,total_images,last_price,total_area,first_day_exposition,rooms,ceiling_height,floors_total,living_area,floor,is_apartment,...,balcony,locality_name,airports_nearest,cityCenters_nearest,parks_around3000,parks_nearest,ponds_around3000,ponds_nearest,days_exposition,locality_name_lem
0,20,13000000.0,108.0,2019-03-07T00:00:00,3,2.70,16,51,8,NaN,...,NaN,санкт-петербург,18863.0,16028.0,1.0,482.0,2.0,755.0,NaN,санкт-петербург \n
1,7,3350000.0,40.4,2018-12-04T00:00:00,1,2.65,11,18,1,NaN,...,2.0,посёлок шушары,12817.0,18603.0,0.0,NaN,0.0,NaN,81.0,шушары \n
2,10,5196000.0,56.0,2015-08-20T00:00:00,2,2.65,5,34,4,NaN,...,0.0,санкт-петербург,21741.0,13933.0,1.0,90.0,2.0,574.0,558.0,санкт-петербург \n
3,0,64900000.0,159.0,2015-07-24T00:00:00,3,2.65,14,45,9,NaN,...,0.0,санкт-петербург,28098.0,6800.0,2.0,84.0,3.0,234.0,424.0,санкт-петербург \n
4,2,10000000.0,100.0,2018-06-19T00:00:00,2,3.03,14,32,13,NaN,...,NaN,санкт-петербург,31856.0,8098.0,2.0,112.0,1.0,48.0,121.0,санкт-петербург \n


All looks fine, just need to adjust data type in `last_pirce` and `total_area` columns.

In [114]:
# changing the data type and cheking if everything is ok
df['last_price'] = df['last_price'].astype(int)
df['total_area'] = df['total_area'].astype(int)

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17453 entries, 0 to 23697
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   total_images          17453 non-null  int64  
 1   last_price            17453 non-null  int64  
 2   total_area            17453 non-null  int64  
 3   first_day_exposition  17453 non-null  object 
 4   rooms                 17453 non-null  int64  
 5   ceiling_height        17453 non-null  float64
 6   floors_total          17453 non-null  int64  
 7   living_area           17453 non-null  int64  
 8   floor                 17453 non-null  int64  
 9   is_apartment          2077 non-null   object 
 10  studio                17453 non-null  bool   
 11  open_plan             17453 non-null  bool   
 12  kitchen_area          17453 non-null  int64  
 13  balcony               9011 non-null   float64
 14  locality_name         17453 non-null  object 
 15  airports_nearest   

The data is neat and clean and ready for further analysis. 

# Data analysis




## Price per sq.m

Next, I will calculate the price per square meter for each apartment and add it to the table. 
 

In [115]:
# aq.m price in sqm_price 
df['sqm_price'] = df['last_price']/df['total_area']

# changing the data type
df['sqm_price'] = df['sqm_price'].astype(int)


On the top of it it would be nice to know day of the week (dow), month and year of publication of the ad.

I will add them to the following columns:

* `dow` - day of the week
* `month` - month
* `year` - year




In [116]:
# converting date to the better format
df['first_day_exposition'] = pd.to_datetime(df['first_day_exposition'], format = '%Y-%m-%dT%H:%M:%S')

# and adding columns
df['dow'] = df['first_day_exposition'].dt.weekday
df['month'] = df['first_day_exposition'].dt.month
df['year']=df['first_day_exposition'].dt.year

# checking the results
df.head(3)

,total_images,last_price,total_area,first_day_exposition,rooms,ceiling_height,floors_total,living_area,floor,is_apartment,...,parks_around3000,parks_nearest,ponds_around3000,ponds_nearest,days_exposition,locality_name_lem,sqm_price,dow,month,year
0,20,13000000,108,2019-03-07,3,2.70,16,51,8,NaN,...,1.0,482.0,2.0,755.0,NaN,санкт-петербург \n,120370,3,3,2019
1,7,3350000,40,2018-12-04,1,2.65,11,18,1,NaN,...,0.0,NaN,0.0,NaN,81.0,шушары \n,83750,1,12,2018
2,10,5196000,56,2015-08-20,2,2.65,5,34,4,NaN,...,1.0,90.0,2.0,574.0,558.0,санкт-петербург \n,92785,3,8,2015


All fine, now I will add the apartment floor by category:

* `1` - groud floor
* `2` - last floor
* `3` - any other floor

In [117]:
# adding floor category to  first_last column

def first_last_func(value):
    if value['floor'] == value['floors_total']:
        return "1"
    elif value['floor'] == 1:
         return "2"
    else:
        return "3"


df['first_last'] = df.apply(first_last_func, axis=1)

# and changing the data type
df['first_last'] = df['first_last'].astype(int)

Let's calculate and add the ratio of living area to total area, as well as the ratio of kitchen area to total area.


In [118]:
# ratio of living area to total area
df['living_to_total_%'] = df['living_area']/df['total_area']*100
# changing the data type to make it easier to work with
df['living_to_total_%'] = df['living_to_total_%'].astype(int)

# ratio of kitchen area to total area
df['kitchen_to_total_%'] = df['kitchen_area']/df['total_area']*100
# changing the data type to make it easier to work with
df['kitchen_to_total_%'] = df['kitchen_to_total_%'].astype(int)